<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install lxml
!pip install parse

In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 25 #limit to check whether they are relative or not

In [12]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> 나쁜형사
2 -> 조병준
3 -> 홍남기
4 -> 더페이스샵
5 -> 처음학교로
6 -> mp그룹
7 -> 땐뽀걸즈
8 -> 남선알미늄
9 -> 2019 수능 등급컷
10 -> 준스피자
11 -> 미스터피자
12 -> 김혜경
13 -> 이영자
14 -> 손태영
15 -> 노니
16 -> 수능 성적 발표
17 -> 수능 등급컷
18 -> 국민대학교
19 -> 조제 호랑이 그리고 물고기들
20 -> 설인아


In [24]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target) + 10*word.count(target)) 
 
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        print(word, ' and ', target,)
        #the representive number should be minimized
        if label[int(cnt1/2)] > label[int(cnt2/2)]:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

조병준  and  준스피자
mp그룹  and  미스터피자
2019 수능 등급컷  and  수능 등급컷
준스피자  and  조병준
수능 성적 발표  and  수능 등급컷


In [25]:
print('\n',label,'\n')

print(matrix)

score =[0]
score = score*20;

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 10, 3, 4, 5, 11, 7, 8, 17, 10, 11, 12, 13, 14, 15, 17, 17, 18, 19, 20] 

[[0, 9, 9, 9, 9, 9, 16, 9, 9, 9, 11, 11, 11, 0, 9, 9, 18, 9, 9, 9], [15, 0, 9, 9, 9, 9, 11, 9, 9, 33, 11, 11, 11, 0, 9, 9, 18, 9, 9, 9], [15, 9, 0, 9, 9, 9, 11, 9, 9, 9, 11, 11, 11, 0, 9, 9, 18, 9, 9, 9], [15, 9, 9, 0, 9, 9, 11, 9, 9, 9, 11, 11, 11, 0, 9, 9, 18, 9, 9, 9], [15, 9, 9, 9, 0, 9, 11, 9, 9, 9, 11, 11, 11, 0, 9, 9, 18, 9, 9, 9], [15, 9, 9, 9, 9, 0, 11, 9, 9, 9, 107, 11, 11, 0, 9, 9, 18, 9, 9, 9], [21, 9, 9, 9, 9, 9, 0, 9, 9, 9, 11, 11, 11, 0, 9, 9, 18, 9, 9, 9], [15, 9, 9, 9, 9, 9, 11, 0, 9, 9, 11, 11, 11, 0, 9, 9, 18, 9, 9, 9], [15, 9, 9, 9, 9, 9, 11, 9, 0, 9, 11, 11, 11, 0, 9, 10, 175, 9, 9, 9], [15, 36, 9, 9, 9, 9, 11, 9, 9, 0, 11, 11, 11, 0, 9, 9, 18, 9, 9, 9], [15, 9, 9, 9, 9, 11, 11, 9, 9, 9, 0, 13, 11, 0, 9, 9, 18, 9, 9, 9], [15, 9, 9, 9, 9, 9, 11, 9, 9, 9, 11, 0, 11, 0, 9, 9, 18, 9, 9, 9], [15, 9, 9, 9, 9, 9, 11, 9, 9, 9, 11, 11, 0, 0, 9, 9, 18, 9, 9, 9], [10, 4, 4, 4, 4, 4, 6, 4, 4, 4, 6, 

In [26]:
#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  나쁜형사
2 .  조병준 | 준스피자
3 .  홍남기
4 .  mp그룹 | 미스터피자
5 .  더페이스샵
6 .  2019 수능 등급컷 | 수능 성적 발표 | 수능 등급컷
7 .  처음학교로
8 .  땐뽀걸즈
9 .  남선알미늄
10 .  김혜경
11 .  이영자
12 .  손태영
13 .  노니
14 .  국민대학교
15 .  조제 호랑이 그리고 물고기들
16 .  설인아
